In [ ]:
"""
--- Day 9: Encoding Error ---
With your neighbor happily enjoying their video game, you turn your 
attention to an open data port on the little screen in the seat in 
front of you.

Though the port is non-standard, you manage to connect it to your 
computer through the clever use of several paperclips. Upon connection,
the port outputs a series of numbers (your puzzle input).

The data appears to be encrypted with the eXchange-Masking Addition System 
(XMAS) which, conveniently for you, is an old cypher with an important weakness.

XMAS starts by transmitting a preamble of 25 numbers. After that, each 
number you receive should be the sum of any two of the 25 immediately 
previous numbers. The two numbers will have different values, and there 
might be more than one such pair.

For example, suppose your preamble consists of the numbers 1 through 25 
in a random order. To be valid, the next number must be the sum of two 
of those numbers:

26 would be a valid next number, as it could be 1 plus 25 (or many other 
pairs, like 2 and 24).
49 would be a valid next number, as it is the sum of 24 and 25.
100 would not be valid; no two of the previous 25 numbers sum to 100.
50 would also not be valid; although 25 appears in the previous 25 numbers, 
the two numbers in the pair must be different.
Suppose the 26th number is 45, and the first number (no longer an option, 
as it is more than 25 numbers ago) was 20. Now, for the next number to be 
valid, there needs to be some pair of numbers among 1-19, 21-25, or 45 that 
add up to it:

26 would still be a valid next number, as 1 and 25 are still within the previous 25 numbers.
65 would not be valid, as no two of the available numbers sum to it.
64 and 66 would both be valid, as they are the result of 19+45 and 21+45 respectively.

Here is a larger example which only considers the previous 5 numbers (and has a preamble of length 5):

35
20
15
25
47
40
62
55
65
95
102
117
150
182
127
219
299
277
309
576

In this example, after the 5-number preamble, almost every number is the sum of two 
of the previous 5 numbers; the only number that does not follow this rule is 127.

The first step of attacking the weakness in the XMAS data is to find the first number 
in the list (after the preamble) which is not the sum of two of the 25 numbers before it.
What is the first number that does not have this property?
"""

In [1]:
import numpy as np
import itertools

In [73]:
## puzzle input

inp_loc = './day_9_input.txt'

puzzle_input = np.loadtxt(inp_loc)

print(puzzle_input.shape)
puzzle_input = puzzle_input.astype('int64')

print("NaN check: {}".format(np.isnan(np.sum(puzzle_input))))

(1000,)
NaN check: False


In [65]:
def find_XMAS_flaws(arr, window, elements=2):
    """
    Accepts array-like object.
    Searches for n-elements in preceeding window
    that add up to current value. Condition: elements
    cannot be same value as each other.
    
    Returns:
    Boolean list of length arr[window:]
    True: flaw found in encryption
    False: passes encryption test
    
    """
    flaws = []
    
    for i in range(len(arr[window:])):
        tmp = []
    
        for C in set(itertools.combinations(arr[i : (window+i)], elements)):
            
            ## !! logic currently only holds for 2 elements
            if int(C[0]) == int(C[1]):
                continue
            
            if ((C[0] + C[1]) == arr[(window+i)]):
                tmp.append(False)
            else:
                tmp.append(True)
                
        if all(x==True for x in tmp):
            flaws.append(True)
        else:
            flaws.append(False)
            
    return flaws

def get_first_flaw(arr, window, elements=2):
    """
    Return the first value that doesn't meet encryption
    """
    flaws = find_XMAS_flaws(arr=arr, window=window, elements=elements)
    
    flaw_idxs = [idx for idx, ele in enumerate(flaws) if ele==True]
    
    try:
        first_flaw = arr[flaw_idxs[0]+window]
        return first_flaw
    except IndexError:
        return None

In [74]:
## test case, answer should be 127

tst_arr = np.array([35,20,15,25,47,40,62,55,65,95,102,117,150,182,127,219,299,277,309,576])

window = 5

# tst_flaws = find_XMAS_flaws(tst_arr, window=window)
# print(tst_flaws)

tst_answer = get_first_flaw(tst_arr, window=window)
print(tst_answer)


127


In [75]:
## answer part 1

window=25


pt1_ans = get_first_flaw(puzzle_input, window=window)

print(pt1_ans)

22406676


In [ ]:
"""
--- Part Two ---
The final step in breaking the XMAS encryption relies on the invalid number 
you just found: you must find a contiguous set of at least two numbers in your 
list which sum to the invalid number from step 1.

Again consider the above example:

35
20
15
25
47
40
62
55
65
95
102
117
150
182
127
219
299
277
309
576
In this list, adding up all of the numbers from 15 through 40 produces the 
invalid number from step 1, 127. (Of course, the contiguous set of numbers 
in your actual list might be much longer.)

To find the encryption weakness, add together the smallest and largest number 
in this contiguous range; in this example, these are 15 and 47, producing 62.

What is the encryption weakness in your XMAS-encrypted list of numbers?
"""

In [84]:
def find_contiguous_sum_group(target_val, arr, group_min=2):
    """
    Find a contiguous slice of an array that
    sums to the target value
    """
    summed_val = None
    arr_length = len(arr)
    
    while summed_val != target_val:
        steps = arr_length - group_min + 1
        for S in range(steps):
            if (S+group_min) != arr_length:
                if sum(arr[S:S+group_min]) == target_val:
                    summed_val = sum(arr[S:S+group_min])
                    summed_arr = arr[S:S+group_min]
            else:
                if sum(arr[S:]) == target_val:
                    summed_val = sum(arr[S:])
                    summed_arr = arr[S:]
        
        group_min += 1
        
    return summed_arr

In [85]:
## answer 2

summed_arr = find_contiguous_sum_group(22406676, puzzle_input)

print(np.amin(summed_arr) + np.amax(summed_arr))

2942387
